In [1]:
include("Axis_Aligned_Box.jl");
type Mondrian_Node
    parent::Nullable{Mondrian_Node}
    left::Nullable{Mondrian_Node}
    right::Nullable{Mondrian_Node}
    τ::Float64
    node_type::Array{Bool,1}        # node,leaf,root
    δ::Nullable{Int}                # split dimension
    ζ::Nullable{Float64}            # split position
    Θ::Nullable{Axis_Aligned_Box}   # data boxes B
    tab::Array{Int}
    c::Array{Int}
    Gₚ::Array{Float64}
end

function Mondrian_Node()
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      0,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64)
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64, node_type::Array{Bool,1})
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,
                      node_type,
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

type Mondrian_Tree
    root::Nullable{Mondrian_Node}
    leaves::Array{Mondrian_Node,1}
end

function Mondrian_Tree()
    T = Mondrian_Tree(Nullable{Mondrian_Node}(),Array{Mondrian_Node,1}())
end

function Mondrian_Tree(N::Mondrian_Node)
    T = Mondrian_Tree(N,Array{Mondrian_Node,1}())
end

Mondrian_Tree

In [2]:
using Distributions

function Sample_Mondrian_Tree(λ,D)
    δ = []
    ζ = []
    n = 1:size(D,1)
    e = Mondrian_Node(0.0,[false,false,true])
    T = Mondrian_Tree(e)
    Θ = Axis_Aligned_Box(get_intervals(D[1]))
    e.Θ = Θ
    e.tab = zeros(size(unique(D[2]),1))
    e.c = zeros(size(unique(D[2]),1))
    e.Gₚ = zeros(size(unique(D[2]),1))
    Sample_Mondrian_Block(e, Θ, λ, δ, ζ, T)
    return T
end

function Sample_Mondrian_Block(j, Θ, λ, δ, ζ, tree)
    E = rand(Exponential(1/Linear_dimension(Θ)))
    if j.node_type[3]==true
        τₚ = 0
    else
        τₚ = (get(j.parent)).τ
    end
    if τₚ + E < λ
        d,x = sample_split_dimension(Θ)
        j.δ = d
        j.ζ = x
        j.τ = τₚ+E
        Θᴸ = copy(Θ)
        Θᴿ = copy(Θ)
        Θᴸ.Intervals[d,2]=x
        Θᴿ.Intervals[d,1]=x
        left = Mondrian_Node(0.0, [true,false,false])
        left.parent = j
        left.Θ = Θᴸ
        left.tab = zeros(size(j.tab))
        left.c = zeros(size(j.tab))
        left.Gₚ = zeros(size(unique(D[2]),1))
        right = Mondrian_Node(0.0, [true,false,false])
        right.parent = j
        right.Θ = Θᴿ
        right.tab = zeros(size(j.tab))
        right.c = zeros(size(j.tab))
        right.Gₚ=zeros(size(unique(D[2]),1))
        j.left = left
        j.right = right
        Sample_Mondrian_Block(left, Θᴸ, λ, δ, ζ, tree)
        Sample_Mondrian_Block(right, Θᴿ,λ, δ, ζ, tree)
    else
        j.τ = λ
        j.node_type = [false,true,false]
        push!(tree.leaves,j)
    end
end

function get_data_indices(Θ::Axis_Aligned_Box, D::Array{Float64,2})
    indices = []
    include = false
    for i in 1:size(D,1)
        for j in 1:size(Θ.Intervals,1)
            if (D[i,j] < Θ.Intervals[j,1] || D[i,j] > Θ.Intervals[j,2])
                include = false
                break
            end
            include = true
        end
        if (include)    
            push!(indices, i)
        end
    end
    return indices
end

function set_label_distribution!(j,Θ, D)
    indices = get_data_indices(Θ, D)
    for k in 1:length(unique(D[2]))
        j.Gₚ = length(D[2][D[2].==k])
    end
    j.Gₚ = j.Gₚ/(sum(j.Gₚ))
end

function initialize_posterior_leaf_counts!(Tree, D)
    X = D[1]
    Y = D[2]
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),X)
        y = Y[indices]
        for k in 1:length(leaf.c)
            leaf.c[k] = length(y[y.==k])
            leaf.tab[k] = min(leaf.c[k],1)
        end
    end
end

function initialize_posterior_counts!(Tree,D)
    initialize_posterior_leaf_counts!(Tree,D)
    for leaf in Tree.leaves
        j = leaf
        while true
            if j.node_type[2]==false
                for k in 1:length(j.c)
                    j.c[k] = get(j.left).tab[k]+get(j.right).tab[k]
                end
            end
            for k in 1:length(j.c)
                j.tab[k] = min(j.c[k],1)
            end
            if j.node_type[3]==true
                break
            else
                j = get(j.parent)
            end
        end
    end
end

function update_posterior_counts!(leaf,y)
    leaf.c[y] += 1
    l = leaf
    while true
        if l.tab[y] == 1
            return
        else
            if l.node_type[2] == false
                l.c[y] = get(l.left).tab[y]+get(l.right).tab[y]
            end
            l.tab[y] = min(l.c[y],1)
            if l.node_type[3] == true
                return
            else
                l = get(l.parent)
            end
        end
    end         
end

function update_posterior_counts_batch!(Tree,D)
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),D[1])
        for y in D[2][indices,:]
            update_posterior_counts(leaf,y)
        end
    end
end

function compute_predictive_posterior_distribution!(Tree, γ=1)
    J = [get(Tree.root)]
    while (size(J,1) != 0)
        j = shift!(J)
        if (j.node_type[3]==true)
            p = ones(length(j.c))./length(j.c)
            d = exp(-γ*(j.τ))
        else 
            d = exp(-γ*(j.τ-get(j.parent).τ))
            p = get(j.parent).Gₚ
        end
        for k in 1:length(j.c)
            j.Gₚ[k] = (1/(sum(j.c)+1))*(j.c[k]-d*j.tab[k]+sum(j.tab)*p[k])
        end
        if j.node_type[2] == false 
            push!(J, get(j.left))
            push!(J, get(j.right))
        end
    end
end

function expected_discount(nⱼ, Δⱼ,γ=1)
    Δ = rand(Truncated(Exponential(1/nⱼ),0,Δⱼ),10000)
    return mean(exp.(-γ*Δ))
end

function predict!(T,x, γ=1)
    j = get(T.root)
    not_sep = 1
    s = zeros(size(j.c,1))
    while true
        if (j.node_type[3] == true)
            Δⱼ = j.τ
        else
            Δⱼ = j.τ - get(j.parent).τ
        end
        nⱼ=0
        for d in size(x,1)
            nⱼ += max(x[d]-get(j.Θ).Intervals[d,2],0) + max(get(j.Θ).Intervals[d,1]-x[d],0)
        end
        pⱼ = 1-exp(Δⱼ*nⱼ)
        if pⱼ > 0
            println("Yes")
            # x branches
            jₓ = Mondrian_Node()
            if (j == get(j.parent).left)
                get(j.parent).left = jₓ
            else
                get(j.parent).right = jₓ
            end
            jₓ.parent = get(j.parent)
            j.parent = jₓ
            jₓ.left = j
            jₓ.right = Mondrian_Node()
            d = expected_discount(nⱼ, Δⱼ, γ)
            for k in 1:length(j.c)
                jₓ.c[k] = min(j.c[k],1)
            end
            jₓ.tab = jₓ.c
            for k in 1:length(jₓ.c)
                jₓ.Gₚ[k] = 1/(sum(jₓ.c)+1)*(jₓ.c[k] - d*jₓ.tab[k]+d*sum(jₓ.tab)*get(jₓ.parent).Gₚ[k])
            end
            for k in 1:length(s)
                s[k] += not_sep*(1-pⱼ)*jₓ.Gₚ[k]
            end
        end
        if j.node_type[2] == true
            for k in 1:length(s)
                s[k] += not_sep*(1-pⱼ)*j.Gₚ[k]
            end
            return s
        else
            not_sep = not_sep*(1-pⱼ)
            if x[get(j.δ)] <= get(j.ζ)
                j = get(j.left)
            else
                j = get(j.right)
            end
        end
    end
end

predict! (generic function with 2 methods)

In [4]:
using MLBase
function Fakedata(n,dim) 
    x = randn(n,dim)
    y = (sum(exp.(x)/(1+exp.(x)),2)).>0.5
    return 1.0*x,1*y[:,1]
end

d=5
X, Y = Fakedata(1000,d);
D = (X,Y.+1)

x,y = Fakedata(100,d)
D_test = (x,y.+1)

([0.65927 -1.04542 … -1.47148 -0.55728; -0.346171 -0.734048 … -0.305209 0.245106; … ; -0.532437 -0.0702832 … 0.53052 0.320615; -1.84471 -0.376782 … -0.856672 0.281011], [2, 1, 2, 1, 1, 1, 1, 1, 2, 2  …  2, 1, 1, 1, 2, 1, 2, 2, 1, 1])

In [5]:
@time MT = Sample_Mondrian_Tree(0.3,D);
initialize_posterior_counts!(MT,D);
compute_predictive_posterior_distribution!(MT,10*size(D[1],2));

pred = []
for i in 1:size(D[1],1) 
    p = predict!(MT,D[1][i,:],10*size(D[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
println("Train accuracy")
println(correctrate(D[2],convert(Array{Int,1},pred)))
println(unique(pred))

x,y = Fakedata(1000,d)
D_test = (x,y.+1)
pred = []
for i in 1:size(D_test[1],1) 
    p = predict!(MT,D_test[1][i,:],10*size(D_test[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
println("Test accuracy")
println(correctrate(D_test[2],convert(Array{Int,1},pred)))
println(unique(pred))

  0.533757 seconds (342.51 k allocations: 18.359 MiB, 0.86% gc time)
Train accuracy
0.742
[1, 2]
Test accuracy
0.729
[2, 1]


In [10]:
function Mondrian_Forest_predict!(D_train,D_pred,λ,nₜ=200)
    trees = Array{Mondrian_Tree}(nₜ)
    pred = Array{Int, 2}(nₜ,size(D_pred,1))
    for i in 1:nₜ
        trees[i] = Sample_Mondrian_Tree(λ,D_train)
        initialize_posterior_counts!(trees[i],D_train);
        compute_predictive_posterior_distribution!(trees[i],10*size(D_train[1],2));
        for j in 1:size(D_pred,1) 
            p = predict!(trees[i],D_pred[j,:],10*size(D_pred,2))
            if p[1] > p[2]
                pred[i,j] = 1
            else
                pred[i,j] = 2
            end
        end
    end
    p = Array{Int,1}(size(D_pred,1))
    for i in 1:size(D_pred,1)
        p[i] = mode(pred[:,i])
    end
    return p 
end
# println("Train accuracy")
# p = Mondrian_Forest_predict!(D,D[1],0.3)
# println(correctrate(p,convert(Array{Int,1},D[2])))
@time p = Mondrian_Forest_predict!(D,D_test[1],0.3,100)
println(correctrate(p,convert(Array{Int,1},D_test[2])))
println("Test accuracy")

  9.870654 seconds (210.03 M allocations: 4.235 GiB, 7.91% gc time)
0.866
Test accuracy


In [204]:
using DecisionTree
forest = build_forest(Y, X, d, 100)

pred = 1*apply_forest(forest,x)
println("Correct: ", correctrate(y, pred))
println("Error: ", errorrate(y,pred))

Correct: 0.905
Error: 0.095


In [104]:
for leaf in MT.leaves
    println(leaf.c)
end

[0, 0]
[0, 0]
[0, 1]
[0, 0]
[38, 0]
[2, 3]
[0, 0]
[6, 5]
[35, 2]
[30, 19]
[0, 0]
[0, 0]
[7, 3]
[394, 182]
[0, 0]
[0, 1]
[0, 1]
[21, 26]
[0, 38]
[1, 45]
[0, 2]
[0, 20]
[0, 25]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 10]
[0, 0]
[0, 1]
[0, 32]
[0, 0]
[0, 0]
[0, 24]
[0, 3]
[0, 21]
[0, 1]


In [206]:
include("load_titanic.jl")
x_train, y_train, x_test, y_test = load();
D = (x_train, y_train.+1)
D_test = (x_test,y_test.+1)
MT = Sample_Mondrian_Tree(0.03,D);
initialize_posterior_counts!(MT,D);
compute_predictive_posterior_distribution!(MT,10*size(D[1],2));

In [489]:
using MLBase
pred = []
for i in 1:size(D_test[1],1) 
    p = predict!(MT,D_test[1][i,:],10*size(D_test[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
println(correctrate(D_test[2],convert(Array{Int,1},pred)))
println(unique(pred))

0.675
[2, 1]


In [287]:
a.Intervals

LoadError: [91mtype Mondrian_Node has no field Intervals[39m

In [43]:
D[1][1,2] < a.Intervals[2,1] || D[1][1,2] > a.Intervals[2,2]

false

In [40]:
D[1,2]

LoadError: [91mMethodError: no method matching getindex(::Tuple{Array{Float64,2},Array{Int64,1}}, ::Int64, ::Int64)[39m

In [22]:
Y = [1,2,2,2,1]

5-element Array{Int64,1}:
 1
 2
 2
 2
 1

In [24]:
size(Y[Y.==1],1)

2

In [16]:
ones(2)

2-element Array{Float64,1}:
 1.0
 1.0

In [17]:
exp(2)

7.38905609893065

In [19]:
sum([1,2,3])

6